## Initializing Spark Environment and Loading Data

In [1]:
import findspark
findspark.init('/home/shahayush954/spark-3.4.1-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

23/08/17 23:43:43 WARN Utils: Your hostname, ubuntu-22 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/08/17 23:43:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/17 23:43:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
data = spark.read.csv('titanic.csv', header=True, inferSchema=True)

In [6]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



## Data Cleaning and Transformations

In [7]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [8]:
filtered_data = data.select(['Survived','Pclass','Sex','Age','SibSp','Parch','Fare', 'Embarked'])

In [10]:
data_without_null = filtered_data.na.drop() ## droping rows if any column is null

In [11]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer)

In [13]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')
#    A        B        C     --> Category
#    0        1        2     --> StringIndexer
# [1,0,0]  [0,1,0]  [0,0,1]  --> One Hot Encoding
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkedIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkedIndex', outputCol='EmbarkedVec')

In [14]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','Age','SibSp','Parch','Fare', 'EmbarkedVec'], outputCol='features')

In [16]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [17]:
lr_reg = LogisticRegression(featuresCol='features', labelCol='Survived')

In [18]:
pipeline = Pipeline(stages=[gender_indexer, embark_indexer, gender_encoder, embark_encoder, assembler, lr_reg])

In [19]:
train_data, test_data = data_without_null.randomSplit([0.7, 0.3])

In [20]:
model = pipeline.fit(train_data)

In [21]:
results = model.transform(test_data)

In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [23]:
my_evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [24]:
results.select(['Survived', 'prediction']).show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [25]:
AUC = my_evaluator.evaluate(results)

In [26]:
AUC

0.7610771227289581